# Homework 1

## 1.1 The earliest psychologists studied the human mind using introspection. Would this be a good way for researchers to gain insight into human cognitive processes for the purpose of modelling those processes?

I think that introspection can be a good method of figuring out some ways that the mind stores and retrieves information, but I don't think that any algorithm will ever be able to come anywhere close to being as intricate and as complex as the human brain is. The human brain does more than just perform a series of actions. In my philosophy class last semester, we talked about act-based and agent based morality. Someone who lives a agent-based moral life doesn't just have a file cabinet full of different scenarios and their corresponding moral responses. Instead, doing the moral thing is simply second nature. They don't have to dig around and find what the "correct" response is. They can just do it.

This is how I think of artificial intelligence. Many people think that there will be robots at some point that will have human emotion and feel and basically be human (just a programmed one). We talked about something similar in my Perspectives in Computing class. We were talking about how this one Christian programmer believes that the ethical thing to do is to build bots that will go into war instead of humans because they can be rational and ethical and aren't swayed by emotions. But how can a robot be able to make a decision like that? There are just so many factors and, sure, it will be safer for the people sending the robots because they won't be sending men into the line of fire, but I don't see how it will be good and successful in the long run.

I think that introspection can help model ASPECTS of human cognitive processes, but I don't believe that it can ever come to the point where a computer can actually fully think and behave like a human does.

## 1.2 Implement a local search formulation of a Traveling Salesperson Problem and use the AIMA hill-climbing and simulated annealing implementations to solve it.

State: a list that shows the path between connecting cities (ie. [city_1, city_2, city_3, city_4, city_5])

Actions: a list of the index of the cities that are to be switched (ie. [index_city_3, index_city_2])

In [108]:
'''
This program implements a local search formulation of a Traveling Salesperson Problem for Homework 1 in CS-344.
The problem is solved using hill-climbing and simulated annealing.
The Traveling Salesperson world is defined as a connected, undirected graph.
This allows the salesperson can go either way between cities).
   
Author: Sarah Whitten (smw42)
Date: February 27, 2020
Course: CS-344-A
Assignment: Homework 1
'''
   
import random
from search import Problem, UndirectedGraph, hill_climbing, simulated_annealing, exp_schedule

class TravelingSalesPerson(Problem):
    
    # initialize
    def __init__(self, initial, graph):
        self.initial = initial
        self.graph = graph
        
    # swap cities in the route list
    def actions(self, state):
        r1 = random.randrange(0, len(state), 1)
        r2 = random.randrange(0, len(state), 1)
        actions = [[r1, r2]]
        return actions
    
    # do the action on a COPY of the board
    def result(self, state, action):
        new_state = state[:]
        temporary = new_state[action[0]]
        new_state[action[0]] = new_state[action[1]]
        new_state[action[1]] = temporary
        return new_state
    
    # was every city visited?
    def test_goal(self, state):
        cities = self.graph.nodes()
        for i in range(len(cities)):
            if cities[i] not in state:
                return False
            elif state[0] != state[-1]:
                return False
            else:
                return True
    
    # find value of the state from the cost of the path
    def value(self, state):
        cost = 0
        for i in range(len(state) - 1):
            if state[i] == state[i + 1]:
                cost += 0
            else:
                cost += self.graph.get(state[i], state[i + 1])
        # multiply it by -1 instead of having to rewrite everything to get the smallest
        return cost * -1
    
# create world graph
my_world = UndirectedGraph(dict(
    A=dict(B=3, C=5, D=7, E=9),
    B=dict(C=2, D=4, E=6),
    C=dict(D=10, E=20),
    D=dict(E=15)))

# connect the route
def connect(graph):
    cities = graph.nodes()
    for i in range(len(cities)):
        for j in range(len(cities)):
            if graph.get(cities[i], cities[j]) == None:
                graph.connect(cities[i], cities[j])
                
my_route = ['E', 'D', 'C', 'B', 'A']
connect(my_world)

problem = TravelingSalesPerson(my_route, my_world)

# solve the problem with hill-climbing
hc_solution = hill_climbing(problem)
print('\nAttempting Hill-Climbing:')
print('\n\tSolution: ' + str(hc_solution))
print('\n\tValue: ' + str(problem.value(hc_solution)))
print('\n\tGoal reached? ' + str(problem.test_goal(hc_solution)))

# solve the problem with simulated annealing
annealing = simulated_annealing(problem, exp_schedule(k=20, lam=0.005, limit=10000))
print('\nAttempting Simulated Annealing:')
print('\n\tSolution: ' + str(annealing))
print('\n\tValue: ' + str(problem.value(annealing)))
print('\n\tGoal reached? ' + str(problem.test_goal(annealing)))


Attempting Hill-Climbing:

	Solution: ['E', 'D', 'C', 'B', 'A']

	Value: -30

	Goal reached? False

Attempting Simulated Annealing:

	Solution: ['D', 'A', 'C', 'B', 'E']

	Value: -20

	Goal reached? False


The hill-climbing method consistently gives me a worse value than the simulated annealing method. However, sometimes the values are not too different. Also, simulated annealing consistently gives me a value of 20, whereas hill-climbing gives a value in the range of -20 to -30. Neither of the methods ever succeed in getting me to my goal state (even after running it a bazillion times). Maybe my_route was too far off of the actual best route for it to reach the goal state. I'm kind of disappointed and I'm not sure if it's not working because of my code or if it's from something else. (Also it's a pretty simple graph that I'm using, so maybe it would be different if things were more complex.)

## 1.3 Formulate a course-scheduling domain and implement it using the AIMA constraint satisfaction framework.

Variables: courses

Values: class time, classroom, and faculty

In [115]:
'''
This program implements the constraint satisfaction problem to create a schedule constraint problem.
   
Author: Sarah Whitten (smw42)
Date: February 27, 2020
Course: CS-344-A
Assignment: Homework 1
'''

from csp import CSP, min_conflicts, parse_neighbors

def schedule():
    
    # create lists of courses, faculty, times, and rooms
    courses = ['CS108', 'CS112', 'CS212', 'CS214', 'CS344', 'CS384']
    faculty = ['Norman', 'Adams', 'Plantinga', 'VanderLinden', 'Schuurman']
    times = ['MWF0900', 'MWF1030', 'TTH1200', 'MWF1330']
    rooms = ['NH253', 'SB382']
    
    # 40 possible combinations because 5 * 4 * 2
    combination = [0] * 40
    i = 0
    
    # make every possible combination, which will later be checked against constraints
    for x in range(len(faculty)):
        for y in range(len(times)):
            for z in range(len(rooms)):
                combination[i] = [faculty[x], times[y], rooms[z]]
                i += 1
                
    domains = {}
    for course in courses:
        domains[course] = combination
        
    neighbors = parse_neighbors("""
    CS108: CS112 CS212 CS214 CS344 CS384;
    CS112: CS212 CS214 CS344 CS384;
    CS212: CS214 CS344 CS384;
    CS214: CS344 CS384;
    CS344: CS384""")
    
    def constraints(A, a, B, b):
        
        # each course is only offered once in the semester
        if A == B:
            return False
        # some courses can only be taught by certain professors
        elif A == 'CS112' and a[0] != 'Adams':
            return False
        elif B == 'CS112' and b[0] != 'Adams':
            return False
        elif A == 'CS212' and a[0] != 'Plantinga':
            return False
        elif B == 'CS212' and b[0] != 'Plantinga':
            return False
        elif A == 'CS344' and a[0] != 'VanderLinden':
            return False
        elif B == 'CS344' and b[0] != 'VanderLinden':
            return False
        elif A == 'CS384' and a[0] != 'Schuurman':
            return False
        elif B == 'CS384' and b[0] != 'Schuurman':
            return False
        # the professor can only teach one class at a time
        elif a[0] == b[0] and a[1] == b[1]:
            return False
        # only one class can be in a room at a time
        elif a[1] == b[1] and a[2] == b[2]:
            return False
        # if none of these things occur, we have found a good schedule!
        else:
            return True
        
    return CSP(courses, domains, neighbors, constraints)

result = min_conflicts(schedule(), max_steps=1000)

if result is None:
    print("No good schedule was found :(")
else:
    print("At least one good schedule was found!")
    print(result)

At least one good schedule was found!
{'CS108': ['Norman', 'MWF1030', 'SB382'], 'CS112': ['Adams', 'TTH1200', 'NH253'], 'CS212': ['Plantinga', 'MWF0900', 'NH253'], 'CS214': ['Plantinga', 'MWF1030', 'NH253'], 'CS344': ['VanderLinden', 'MWF1330', 'NH253'], 'CS384': ['Schuurman', 'TTH1200', 'SB382']}
